# Symmetric cryptography using PyNaCl

We are going to use libsodium library to do a practical example of symmetric cipher. First, Alice and Bob establish a common random key using cryptographically secure pseudorandom generators, in this example we skip the Diffie-Hellman key exchange (shown already) and generate a fresh key using the system random.

## Step 1: Generating key using Diffie-Hellman

Alice and Bob need to perform DH exchange protocol to get the key. Here for simplicity we'll generate a random key of 32 bytes with the system random.

In [1]:
import secrets

r = secrets.SystemRandom()

# our key is going to be 32 bytes or larger
key_len = 32

key_int = r.getrandbits(8*key_len)
key_bytes = key_int.to_bytes(key_len, 'big')

In [2]:
print("key in different basis:\n")

print(f"(bytes):\n{key_bytes}\n")
print(f"(int):\n{key_int}\n")
print(f"(binary):\n{bin(key_int)[2:]}\n")
print(f"(individual bytes in integer)\n{[k for k in key_bytes]}")

key in different basis:

(bytes):
b'\xf6\x8fs\xa0u\xc8-\xfb\x90\x954f\xf3m\xd9\xed\xa1\xe8o\x92\xea\xa6\xee|\x97\x96\xf3\xc1\xe3\x03J\x08'

(int):
111522417908572075402303016172274297589925356449568312866119151854730992634376

(binary):
1111011010001111011100111010000001110101110010000010110111111011100100001001010100110100011001101111001101101101110110011110110110100001111010000110111110010010111010101010011011101110011111001001011110010110111100111100000111100011000000110100101000001000

(individual bytes in integer)
[246, 143, 115, 160, 117, 200, 45, 251, 144, 149, 52, 102, 243, 109, 217, 237, 161, 232, 111, 146, 234, 166, 238, 124, 151, 150, 243, 193, 227, 3, 74, 8]


## Use PyNaCl to secure communication

In [3]:
import nacl.secret as secret
import nacl.utils as utils

# define the box, an object to encrypt/decrypt
box = secret.SecretBox(key_bytes)

In [4]:
# the message one wants to send

message = b"How much wood would a woochuck chuck if a woodchuck...?"

In [5]:
# nonce (number used once) can be public but never reused for the same key
nonce = utils.random(secret.SecretBox.NONCE_SIZE)

# encryption with the message and the nonce
ciphertext = box.encrypt(message, nonce)

In [6]:
print(f"secet_key:\n{key_bytes}\n")
print(f"nonce:\n{nonce}\n")
print(f"ciphertext:\n{ciphertext.ciphertext}\n")

decrypted_message = box.decrypt(ciphertext)
print(f"recovered secret:\n{decrypted_message}\n")

secet_key:
b'\xf6\x8fs\xa0u\xc8-\xfb\x90\x954f\xf3m\xd9\xed\xa1\xe8o\x92\xea\xa6\xee|\x97\x96\xf3\xc1\xe3\x03J\x08'

nonce:
b'6\x01\xc3h4\x9a\xa7\xa4N=\xf5_*J\xb1uL\xa4\xac\x83\x8ai\xd0\x16'

ciphertext:
b'" \xcc\x05\xc3\xa6\x1f\x1c\x8a\xb14N\x11v \xb9\xaePK>\xaa\xde\xcb8\x04\xef>/%0\x86}\xfa\x1d\xdb\x9d\x10\xd5\xfd\x18aUf:Xp\xd5\nk\xd1\x98K\xeb\xb2\xe4+\xa1\xf8\xa2\xb6\x1aD\xad.\xdc\xb6\xac\x7f\xcc\xc1.'

recovered secret:
b'How much wood would a woochuck chuck if a woodchuck...?'



In [7]:
assert len(ciphertext.ciphertext)==len(message) + box.MACBYTES

## Other libraries, cryptogrhapy and Fernet encryption

In [19]:
from cryptography.fernet import Fernet

message = b"aabbc"
# secret key generation
secret_key = Fernet.generate_key()

print(f"secret key:\n\t{secret_key}")
print(f"message:\n\t{message}")

encrypt_decrypt = Fernet(secret_key)
ciphertext = encrypt_decrypt.encrypt(message)
print(f"ciphertext:\n\t{ciphertext}")

recovered_message = encrypt_decrypt.decrypt(ciphertext)
print(f"recovered message:\n\t{recovered_message}")

secret key:
	b'AbxDtYJXuTw3xwoX_ZbFxLnf6OqveoE1OLNfRmzbuhg='
message:
	b'aabbc'
ciphertext:
	b'gAAAAABgJREs_pN04OFlpgExVaBgnmYbCuYJTd51aW_VT8RhgmviEq6Ckgsw1_PSbWxb61Turl9H6Ry-8DldMlx6291RT8H5ZQ=='
recovered message:
	b'aabbc'
